In [0]:
# Load model of support ticket

In [0]:
# Mounting Google Drive locally

from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
saveModelPath = '/content/gdrive/My Drive/SupportTicketData'

In [0]:
ls -la '/content/gdrive/My Drive/SupportTicketData'

total 61323
-rw------- 1 root root   245630 Apr 21 12:48 CountVectorizerModel.pkl
-rw------- 1 root root      353 Apr 21 12:48 labelEncorderModel.pkl
-rw------- 1 root root 62394805 Apr 21 12:49 support_ticket_model.pkl
-rw------- 1 root root   153213 Apr 22 11:13 test.csv


In [0]:
# Import required models

import pickle
import numpy as np
import pandas as pd

In [0]:
# Read split test set
test_df = pd.read_csv(saveModelPath+'/test.csv')

In [0]:
test_df.head()

,Unnamed: 0,Description,Category
0,1294,re secured area tower approved thank officer n...,User Maintenance
1,670,re please tickets assign tickets please trees ...,Database
2,1933,door open floor tower dear could you please fi...,Security
3,1203,re card va se,User Maintenance
4,65,old hi please old breakdown thank officer,Application


In [0]:
# Load CountVectorizer Model

loadCountVectorizer = pickle.load(open(saveModelPath+'/CountVectorizerModel.pkl', 'rb'))

In [0]:
# load the model

loaded_model = pickle.load(open(saveModelPath+'/support_ticket_model.pkl', 'rb'))

In [0]:
# Load label encorder model

loadLabelencorder = pickle.load(open(saveModelPath+'/labelEncorderModel.pkl', 'rb'))


In [0]:
# New mail Eg.

mail = "Internet Issue"

In [0]:
# Create a function to check new value from saved model.

def newValueCheck(text):
    x_val = loadCountVectorizer.transform([text])
    # transform:- Extract token counts out of raw text documents using the vocabulary fitted with fit or the one provided to the constructor.
    # transform:- Transform documents to document-term matrix.
    x_val = x_val.toarray()

    # Predict text
    model_output = loaded_model.predict(x_val)
    # print('Model output:- ', model_output)

    # inverse_transform label encorder
    result = loadLabelencorder.inverse_transform(model_output)
    # inverse_transform:- Transform labels back to original encoding.

    return result[0]

In [0]:
newValueCheck(mail)

'Network'

In [0]:
# Create new colum for Predicted values
test_df['Predictions'] = test_df['Description'].apply(newValueCheck)

In [0]:
test_df.head()

,Unnamed: 0,Description,Category,Predictions
0,1294,re secured area tower approved thank officer n...,User Maintenance,User Maintenance
1,670,re please tickets assign tickets please trees ...,Database,Application
2,1933,door open floor tower dear could you please fi...,Security,Network
3,1203,re card va se,User Maintenance,User Maintenance
4,65,old hi please old breakdown thank officer,Application,Application


In [0]:
# Verify Category and Predition colum.
test_df['Verify'] = test_df['Category'] == test_df['Predictions']

In [0]:
test_df.head()

,Unnamed: 0,Description,Category,Predictions,Verify
0,1294,re secured area tower approved thank officer n...,User Maintenance,User Maintenance,True
1,670,re please tickets assign tickets please trees ...,Database,Application,False
2,1933,door open floor tower dear could you please fi...,Security,Network,False
3,1203,re card va se,User Maintenance,User Maintenance,True
4,65,old hi please old breakdown thank officer,Application,Application,True


In [0]:
totalValue = test_df.shape[0]
trueValue = (test_df['Verify'] == True).sum()
falseValue = (test_df['Verify'] == False).sum()

print('Total False value is {} and True value is {} out of total test value {}.'.format(falseValue, trueValue, totalValue))

Total False value is 181 and True value is 419 out of total test value 600.


In [0]:
# pandas groupby
test_df.groupby(('Verify')).count()

,Unnamed: 0,Description,Category,Predictions
Verify,,,,
False,181,181,181,181
True,419,419,419,419
